#### Generate set of (A,B)

In [2]:
import numpy as np
from generate_perturbed import generate_perturbed

A0 = np.array([[0, 0, 0, -0.048],
               [1, 0, 0, 0.676],
               [0, 1, 0, -2.21],
               [0, 0, 1, 2.6]])
B0 = np.array([[-0.12], [0.68], [-1.4], [1]])

n_states = 4
n_plants = 2048

A_list = generate_perturbed(A0, n_plants, 0.01)
B_list = generate_perturbed(B0, n_plants, 0.005)

A_mat = np.stack(A_list, axis=2)
B_mat = np.stack(B_list, axis=2)

A_uncertain = np.array([[False, False, False, True],
                         [False, False, False, True],
                         [False, False, False, True],
                         [False, False, False, True]])

B_uncertain = np.array([[True], [True], [True], [True]])


#### Qhull

In [7]:
import numpy as np
from scipy.spatial import ConvexHull
import time

In [8]:
tic = time.time()

# Extract the uncertain entries of {(A,B)}
n_unc_A = np.sum(A_uncertain)
n_unc_B = np.sum(B_uncertain)
A_unc_entries = np.zeros((n_plants, n_unc_A))
B_unc_entries = np.zeros((n_plants, n_unc_B))

for i in range(n_plants):
    A_unc_entries[i, :] = A_list[i][A_uncertain]
    B_unc_entries[i, :] = B_list[i][B_uncertain]

# Compute hull
unc_entries = np.hstack((A_unc_entries, B_unc_entries))
hull = ConvexHull(unc_entries)
hull_idx = np.unique(hull.vertices)  # Unique vertices to avoid duplicate simplices
verts = unc_entries[hull_idx, :]
n_AB = len(verts)

print("> convhull() complete")
print(f"> Number of constraints reduced from {n_plants} to {n_AB}")

toc = time.time()
print(f"> Elapsed time: {toc - tic:.4f} seconds")


> convhull() complete
> Number of constraints reduced from 2048 to 817
> Elapsed time: 18.1594 seconds


In [9]:
tic = time.time()

# Extract uncertain entries directly using NumPy advanced indexing
A_unc_entries = np.array([A[A_uncertain] for A in A_list])
B_unc_entries = np.array([B[B_uncertain] for B in B_list])

# Compute convex hull only for unique uncertain entries
unc_entries = np.hstack((A_unc_entries, B_unc_entries))
hull = ConvexHull(unc_entries, qhull_options="QJ")  # QJ helps with precision & speed
verts = unc_entries[hull.vertices, :]
n_AB = len(verts)

print("> convhull() complete")
print(f"> Number of constraints reduced from {n_plants} to {n_AB}")

toc = time.time()
print(f"> Elapsed time: {toc - tic:.4f} seconds")


> convhull() complete
> Number of constraints reduced from 2048 to 817
> Elapsed time: 13.8889 seconds


#### Clarkson's algorithm, hull.exe

In [10]:
# A_unc_entries = np.array([A[A_uncertain] for A in A_list])
# B_unc_entries = np.array([B[B_uncertain] for B in B_list])
# unc_entries = np.hstack((A_unc_entries, B_unc_entries))

# n_dimensions = unc_entries.shape[1]
# n_points = unc_entries.shape[0]
# output = ""

# with open("C:\\users\\conor\\hull\\input.txt", "w") as file:
#     for point in unc_entries:
#         file.write(" ".join(f"{value:.8f}" for value in point) + "\n")
# print(f"File input.txt has been created successfully.")

#### MVEE

[]